In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import scipy.optimize as optimize
import datetime

# Sloan Digital Sky Survey

The Sloan Digital Sky Survery ([Project page](https://www.sdss.org/), [Wikipedia](https://en.wikipedia.org/wiki/Sloan_Digital_Sky_Survey)) truly changed we way the understand the universe.  

Over the course of 20 years, SDSS observed 35% of the sky and cataloged about 1 billion stars and galaxies.

It addition to taking images of such a large part of the sky, SDSS also measure the spectrum of the light from over 4 million objects.  

The spectra are obtained by feeding an individual optical fibre for each target through a hole drilled in an aluminum plate.  The light from the fiber is then passed into a diffraction grating to seperate out the different wavelength so that the spectrum for that target can be measured.   The diffracted light was the directed to an array of sensors, so that each sensor measured the amount of light at a different wavelenght.

Each hole is positioned specifically for a selected target, so every field in which spectra are to be acquired requires a unique plate.  In spectroscopic mode, the telescope tracks the sky in the standard way, keeping the objects focused on their corresponding fibre tips. 

Here is a picture of one such aluminum plate:

![SDSS plug plate](figures/plate-sdss.jpg)

We are going to be looking at the data from one fiber, for one plate.  

By comparing the wavelength of atomic emission lines observed from our target to the known wavelengths of those lines we can measure the doppler shift of the light from that target.

In [ ]:
data = np.loadtxt(open("../data/sds_galaxy.txt", 'rb'), usecols=range(4))

# This is how we pull out the data from columns in the array.

# They put the data in Angstroms, lets use nano-meters instead, 1 Angstrom = 0.1 nm. 
wavelength = data[:,0] / 10.
flux = data[:,1]
best_fit = data[:,2]
sky_flux = data[:,3]

FWIW, the "flux" is actually the amount of energy arriving in a narrow wavelength band, per area, per time, and is given in units of $\frac{{\rm erg}}{{\rm cm}^{2}s^{1}{{\rm angstrom}}^{1}}$.  

Rather that write that out again and again, let's call those "arcane units".

In short you can think of it as the amount of energy coming in from light of a given wavelength.

In [ ]:
_ = plt.plot(wavelength, best_fit)
_ = plt.xlabel(r"$\lambda [nm]$")
_ = plt.ylabel("Flux [arcane units]")
_ = plt.title("A spectrum from SDSS")

### $H\alpha$ line

The brightest line is this spectrum is the Hydrogen $\alpha$ line, which is typicall the brightest line in the optical range. [Wikipedia](https://en.wikipedia.org/wiki/H-alpha)

Let's do a quick estimate of the Wavelength of the H $\alpha$ line from this object, it's redshift, and the implied velocity.  

In [ ]:
peak_idx = np.argmax(flux)
peak_wl = wavelength[peak_idx]
print("The peak wavelength is %0.2f nm" % peak_wl)
H_alpha = 656.4614
print("In a vacuum, the H alpha line is at %.4f nm" % H_alpha)
z1 = (peak_wl/H_alpha)-1
print("This corresponds to a redshift of %.5f" % z1)
r2 = (peak_wl/H_alpha)**2
beta = (1 - (1/r2))/(1 + (1/r2))
print("Based on this line we estimate that this object is moving away from us with a velocity of %0.4f c" % beta)

# Question for dicussion

4.1 About far away is this object based on Hubble's law $v = H_0 d$?

You can use the Hubble parameter $H_0 = 70 {\rm km} {\rm s}^{-1} {\rm Mpc}^{-1}$.

The speed of light is about $c = 3 \times 10^8 m/s$.

$1 {\rm pc} = 3.26 {\rm ly}$.



### Zoom in on $H \alpha$ line.

If we zoom in on the line, we see it is actually three lines.   That complicates things for trying to fit the lines to a model, as we would have to include all three lines in our model.  

In [ ]:
cutout_wl = wavelength[peak_idx-50:peak_idx+50]
cutout_flux = flux[peak_idx-50:peak_idx+50]
_ = plt.plot(cutout_wl, cutout_flux)
_ = plt.xlabel(r"$\lambda [nm]$")
_ = plt.ylabel("Flux [arcane units]")

### The $H \beta$ line.

The brightest line just around 500 nm is the $H \beta$ line, the little sibling of the $H \alpha$ line.  It is more isolated, making it a better choice for us to fit with a simple model.


In [ ]:
mask = (wavelength > 500) & (wavelength < 510)
cutout_wl = wavelength[mask]
cutout_flux = flux[mask]

In [ ]:
_ = plt.plot(cutout_wl, cutout_flux)
_ = plt.xlabel(r"$\lambda [nm]$")
_ = plt.ylabel("Flux [arcane units]")

# Our model

We are going to fit this line to a model with two parts.

   1. We will model this line itself as a Gaussian $ G(\lambda | \mu, \sigma)$ times a prefactor that give the height of the $n$.  
   2. We will model the background as a slope and offset.  To reduce the correlation between the two, we will define the offset at 500 nm.   $p_0 + (\lambda - 500)*p_1$.
   
This gives us a model with five parameters. $(n, \mu, \sigma, p_0, p_1)$.

$m(\lambda | n, \mu, \sigma, p_0, p_1) = n G(\lambda | \mu, \sigma) + p_0 + (\lambda - 500)*p_1$.

This next cell codes up the model and the functions we need for the fitting.

In [ ]:
from functools import partial

def Gauss(x, prefact, mu, sigma):
    return prefact*stats.norm(loc=mu, scale=sigma).pdf(x)

def poly1(x, offset, slope):
    return offset + (x-500)*slope

def model_func(x, prefact, mu, sigma, offset, slope):
    return Gauss(x, prefact, mu, sigma) + poly1(x, offset, slope)

def generic_chi2(params, data_vals, model, x):
    model_vals = model(x, *params)
    return np.sum((data_vals - model_vals)**2)

def cost_func(data_vals, model, x):
    return partial(generic_chi2, data_vals=data_vals, model=model, x=x)

### Initial guess

Let's guess some initial values for the parameters and plot the model to see how it looks.

In [ ]:
prefact_0 = 235.
mu_0 = 505.8
sigma_0 = 0.25
offset_0 = 60
slope_0 = 0

init_pars = (prefact_0, mu_0, sigma_0, offset_0, slope_0)
model_vals = model_func(cutout_wl, *init_pars)

_ = plt.plot(cutout_wl, cutout_flux, label="data")
_ = plt.plot(cutout_wl, model_vals, label="model")
_ = plt.xlabel(r'$\lambda [nm]$')
_ = plt.ylabel(r'Flux [arcane units]')
_ = plt.legend()

# Getting the cost function

First we are going to fit using a cost function that used the full model.  

In [ ]:
our_cost_func = cost_func(cutout_flux, model_func, cutout_wl)
our_cost_func(init_pars)

### Fitting the model

Let's go ahead and fit the model and print out the results.

In [ ]:
result = optimize.minimize(our_chi2_func, x0=np.array(init_pars))
fit_pars = result['x']
cov = result['hess_inv']
model_fit = model_func(cutout_wl, *fit_pars)
background_fit = poly1(cutout_wl, fit_pars[3], fit_pars[4])
print("Best Fit ---------")
print("Line Intensity: %.1f +- %.1f [arcane units]" % (fit_pars[0], np.sqrt(cov[0,0])))
print("Line Peak: %.4f +- %.4f [nm]" % (fit_pars[1], np.sqrt(cov[1,1])))
print("Line Width: %.4f +- %.4f [nm]" % (fit_pars[2], np.sqrt(cov[2,2])))
print("Background at 500 nm: %.2f +- %.2f [arcane units]" % (fit_pars[3], np.sqrt(cov[3,3])))
print("Background slope: %.2f +- %.2f [arcane_units / nm]" % (fit_pars[4], np.sqrt(cov[4,4])))

### Plotting the fit results

Let's make a plot of the data and the model, and let's break out the background part of the model.

In [ ]:
_ = plt.plot(cutout_wl, cutout_flux, label="data")
_ = plt.plot(cutout_wl, background_fit, label="background model")
_ = plt.plot(cutout_wl, model_fit, label="full model")
_ = plt.xlabel(r"$\lambda [nm]$")
_ = plt.ylabel("Flux [arcane units]")
_ = plt.legend()

In [ ]:
H_beta = 486.2721

z2 = (fit_pars[1]/H_beta)-1
print("We fit the peak of the line at %.4f nm" % fit_pars[1])
print("In a vacuum, the H beta line is at %.4f nm" % H_beta)
print("This corresponds to a redshift of %.5f" % z2)
r2 = (fit_pars[1]/H_beta)**2
beta = (1 - (1/r2))/(1 + (1/r2))
print("Based on this line we estimate that this object is moving away from us with a velocity of %0.4f c" % beta)

# Questions for discussion


### How to do a bad fit, version 1, using a bad model.

In this case we are going to forget to include background model.   To do that we are going to construct the cost function using just the Gaussian instead of the full model.

In [ ]:
bad_chi2_func = cost_func(cutout_flux, Gauss, cutout_wl)

init_pars_bad = (prefact_0, mu_0, sigma_0)
model_fit_bad = Gauss(cutout_wl, *fit_pars_bad)
_ = plt.plot(cutout_wl, cutout_flux)
_ = plt.plot(cutout_wl, model_fit_bad)
_ = plt.xlabel(r"$\lambda [nm]$")
_ = plt.ylabel("Flux [arcane units]")

In [ ]:
result_bad = optimize.minimize(bad_chi2_func, x0=np.array(init_pars_bad))
print(result_bad)
model_fit_bad = Gauss(cutout_wl, *result_bad['x'])
_ = plt.plot(cutout_wl, cutout_flux)
_ = plt.plot(cutout_wl, model_fit_bad)
_ = plt.xlabel(r"$\lambda [nm]$")
_ = plt.ylabel("Flux [arcane units]")

# Questions for discussion

5.1 What happened?  How would you interpret what the fit result?  Do the parameters make sense?

### How to do a bad fit, version 2, using bad initial parameters.

In this case we are going to pick bad values for our initial guess.


In [ ]:
prefact_0 = 240.
mu_0 = 505.8
sigma_0 = 0.25
offset_0 = 60
slope_0 = 0

init_pars_bad_2 = (prefact_0, mu_0+2.0, sigma_0, offset_0, slope_0)
model_fit_bad = model_func(cutout_wl, *init_pars_bad_2)

_ = plt.plot(cutout_wl, cutout_flux, label='data')
_ = plt.plot(cutout_wl, model_fit_bad, label='model')
_ = plt.xlabel(r"$\lambda [nm]$")
_ = plt.ylabel("Flux [arcane units]")
_ = plt.legend()

In [ ]:
result_bad_2 = optimize.minimize(our_chi2_func, x0=np.array(init_pars_bad_2))
fit_pars_bad_2 = result_bad_2['x']
model_fit_bad_2 = model_func(cutout_wl, *fit_pars_bad_2)

_ = plt.plot(cutout_wl, cutout_flux)
_ = plt.plot(cutout_wl, model_fit_bad_2)
_ = plt.xlabel(r"$\lambda [nm]$")
_ = plt.ylabel("Flux [arcane units]")

print(result_bad_2)

# Questions for discussion

6.1 What happened?  How would you interpret what the fit result?  Do the parameters make sense?